In [2]:
# ----------------------------------------------
# Load dataset into PostgreSQL using SQLAlchemy
# ----------------------------------------------

import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load environment variables
load_dotenv()

# Database connection details
DB_HOST = os.getenv("RENDER_DB_HOST")
DB_PORT = os.getenv("RENDER_DB_PORT")
DB_NAME = os.getenv("RENDER_DB_NAME")
DB_USER = os.getenv("RENDER_DB_USER")
DB_PASSWORD = os.getenv("RENDER_DB_PASSWORD")

# Create SQLAlchemy engine for PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Verify connection
with engine.connect() as conn:
    print("✅ Connected to PostgreSQL via SQLAlchemy")


csv_path = r"D:\capstone_project_2nd\Luxury_Housing_Sales_Analysis.csv" 
df = pd.read_csv(csv_path)

# Clean column names for SQL 
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")

create_queries = [
    """
    CREATE TABLE Luxury_housing (
        Micro_Market TEXT,
        Project_Name INT, 
        Developer_Name TEXT,
        Unit_Size_Sqft DECIMAL,
        Configuration TEXT,
        Ticket_Price_Cr DECIMAL,
        Transaction_Type TEXT,
        Buyer_Type TEXt,
        Purchase_Quarter DATE,
        Connectivity_Score DECIMAL,
        Amenity_Score DECIMAL,
        Possession_Status TEXT,
        Sales_Channel TEXT,
        NRI_Buyer TEXT,
        Locality_Infra_Score DECIMAL,
        Avg_Traffic_Time_Min INT,
        Price_per_Sqft DECIMAL,
        Quarter_Number INT,
        Year DATE,
        Booking_Flag INT
    )
    """
    ]
print(" New tables created successfully!")

  # inserts multiple rows per statement 
df.to_sql('luxury_housing',con=engine,if_exists='replace',index=False,method='multi',chunksize=5000)    


print(f"✅ Data inserted successfully into table '{'Luxury_housing'}'!")


✅ Connected to PostgreSQL via SQLAlchemy
 New tables created successfully!
✅ Data inserted successfully into table 'Luxury_housing'!


In [3]:
#  3.1 Row count
query_count = "SELECT COUNT(*) AS total_rows FROM luxury_housing;"
count_df = pd.read_sql(query_count, engine)
print("\n Total Rows:")
print(count_df)

# 3.2 Group by possession / booking status
query_group = """
SELECT 
    possession_status AS booking_status, 
    COUNT(*) AS total_transactions
FROM luxury_housing
GROUP BY possession_status
ORDER BY total_transactions DESC;
"""
group_df = pd.read_sql(query_group, engine)
print("\n Group by Booking / Possession Status:")
print(group_df)

# 3.3 Average Ticket Price per Developer
query_avg = """
SELECT 
    developer_name, 
    ROUND(AVG(ticket_price_cr::NUMERIC), 2) AS avg_ticket_price_cr
FROM luxury_housing
GROUP BY developer_name
ORDER BY avg_ticket_price_cr DESC
LIMIT 10;
"""
avg_df = pd.read_sql(query_avg, engine)
print("\n Average Ticket Price per Builder:")
print(avg_df)



 Total Rows:
   total_rows
0      101000

 Group by Booking / Possession Status:
       booking_status  total_transactions
0  Under construction               33769
1              Launch               33641
2       Ready to move               33590

 Average Ticket Price per Builder:
      developer_name  avg_ticket_price_cr
0              Sobha                12.87
1  Total Environment                12.81
2         L&T Realty                12.78
3             Godrej                12.76
4        Puravankara                12.74
5                RMZ                12.74
6           Prestige                12.71
7       Tata Housing                12.67
8            SNN Raj                12.62
9            Brigade                12.61


In [4]:
pd.read_sql("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'luxury_housing';", engine)


,column_name,data_type
0,booking_flag_edits,bigint
1,ticket_price_cr,double precision
2,connectivity_score,double precision
3,amenity_score,double precision
4,locality_infra_score,double precision
5,avg_traffic_time_min,bigint
6,price_per_sqft,double precision
7,quarter_number,bigint
8,year,bigint
9,booking_flag,bigint
